In [189]:
import pandas as pd
import pyarrow.parquet as pq
import datetime


## Get some data to experiment

I extract one series from the train set, i need later to change the code to go series by series, otherwise mistakes will occure.

In [190]:
file_path = '/Users/ra/Library/CloudStorage/OneDrive-HochschuleLuzern/AICH/data/Prepared/V2/train_20231021'

table = pq.read_table(file_path)

# Convert the table to a Pandas DataFrame
df = table.to_pandas()

In [191]:
df.head(3)

,step,anglez,enmo,hour,minute,seconds,day,month,year,seconds_from_midnight,awake,wearable_on,series_id,onset,wakeup
25764840,0,-89.497597,0.0182,11,0,0,6,8,2018,39600,1.0,0.0,2fc653ca75c7,0,0
25764841,1,-89.497398,0.0182,11,0,5,6,8,2018,39605,1.0,0.0,2fc653ca75c7,0,0
25764842,2,-89.495598,0.0180,11,0,10,6,8,2018,39610,1.0,0.0,2fc653ca75c7,0,0


In [192]:
series_list =df.series_id.unique().tolist()

In [193]:
series_list[:2]

['2fc653ca75c7', '31011ade7c0a']

only taking to series for experimenting

In [194]:
sample = df[df.series_id.isin(series_list[1:2])]

sample.head(3)

,step,anglez,enmo,hour,minute,seconds,day,month,year,seconds_from_midnight,awake,wearable_on,series_id,onset,wakeup
26181720,0,-82.680603,0.0,16,45,0,15,8,2017,60300,1.0,0.0,31011ade7c0a,0,0
26181721,1,-82.680603,0.0,16,45,5,15,8,2017,60305,1.0,0.0,31011ade7c0a,0,0
26181722,2,-82.680603,0.0,16,45,10,15,8,2017,60310,1.0,0.0,31011ade7c0a,0,0


In [195]:
sample.series_id.unique()

array(['31011ade7c0a'], dtype=object)

In [196]:
sample.awake.unique()

array([1., 0.])

In [197]:
sample.wearable_on.unique()

array([0., 1.])

I have to recreat the timestamp, maybe we fix this an keep the timestamp for the future or so?

In [198]:
# Convert numeric columns to strings
sample.loc[:, 'hour'] = sample['hour'].astype(str).str.zfill(2)
sample.loc[:, 'minute'] = sample['minute'].astype(str).str.zfill(2)
sample.loc[:, 'seconds'] = sample['seconds'].astype(str).str.zfill(2)
sample.loc[:, 'day'] = sample['day'].astype(str).str.zfill(2)
sample.loc[:, 'month'] = sample['month'].astype(str).str.zfill(2)
sample.loc[:, 'year'] = sample['year'].astype(str)

# Combine the columns into a new column 'timestamp'
sample['timestamp'] = sample['year'] + '-' + sample['month'] + '-' + sample['day'] + ' ' + sample['hour'] + ':' + sample['minute'] + ':' + sample['seconds']

# Convert 'timestamp' to datetime format
sample['timestamp'] = pd.to_datetime(sample['timestamp'])


/var/folders/40/3xwnc19j5wddty6wnqjyxg1h0000gn/T/ipykernel_3536/1806208151.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['timestamp'] = sample['year'] + '-' + sample['month'] + '-' + sample['day'] + ' ' + sample['hour'] + ':' + sample['minute'] + ':' + sample['seconds']
/var/folders/40/3xwnc19j5wddty6wnqjyxg1h0000gn/T/ipykernel_3536/1806208151.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['timestamp'] = pd.to_datetime(sample['timestamp'])


In [199]:
sample.head(3)

,step,anglez,enmo,hour,minute,seconds,day,month,year,seconds_from_midnight,awake,wearable_on,series_id,onset,wakeup,timestamp
26181720,0,-82.680603,0.0,16,45,00,15,08,2017,60300,1.0,0.0,31011ade7c0a,0,0,2017-08-15 16:45:00
26181721,1,-82.680603,0.0,16,45,05,15,08,2017,60305,1.0,0.0,31011ade7c0a,0,0,2017-08-15 16:45:05
26181722,2,-82.680603,0.0,16,45,10,15,08,2017,60310,1.0,0.0,31011ade7c0a,0,0,2017-08-15 16:45:10


i only keep necessary columns for the moment

In [200]:
sample = sample.reset_index()
sample= sample[['step', 'awake', 'wearable_on', 'series_id', 'timestamp']]
sample.head(3)

,step,awake,wearable_on,series_id,timestamp
0,0,1.0,0.0,31011ade7c0a,2017-08-15 16:45:00
1,1,1.0,0.0,31011ade7c0a,2017-08-15 16:45:05
2,2,1.0,0.0,31011ade7c0a,2017-08-15 16:45:10


In [205]:
sample.awake.unique()

array([1., 0.])

In [204]:
sample.wearable_on.unique()

array([0., 1.])

## What happens with removed device timesteps...?

TO DO: restore the lost information of device removed
For the moment i work if i already had done this

## Encode wearable not on

For device not worn i give number 3 to awake, maybe i need smarter solution later on.

In [207]:
sample.loc[sample['wearable_on'] == 0.0, 'awake'] = 3

In [208]:
sample.awake.unique()

array([3., 1., 0.])

In [209]:
sample.head()

,step,awake,wearable_on,series_id,timestamp
0,0,3.0,0.0,31011ade7c0a,2017-08-15 16:45:00
1,1,3.0,0.0,31011ade7c0a,2017-08-15 16:45:05
2,2,3.0,0.0,31011ade7c0a,2017-08-15 16:45:10
3,3,3.0,0.0,31011ade7c0a,2017-08-15 16:45:15
4,4,3.0,0.0,31011ade7c0a,2017-08-15 16:45:20


## Step 1 binning the data on 1 min

In [218]:
# gets the most often occured value
custom_agg_function = lambda x: x.value_counts().idxmax()

# bin the serie dataframe
bin_size = '1Min'
new = sample.resample(bin_size, on='timestamp').agg({
    'series_id': 'first',
    'step': 'first',
    'awake': custom_agg_function,
    'wearable_on': custom_agg_function
}).reset_index()

In [219]:
# sanity check
new[new.step.isna()]

,timestamp,series_id,step,awake,wearable_on


In [220]:
new.head()

,timestamp,series_id,step,awake,wearable_on
0,2017-08-15 16:45:00,31011ade7c0a,0,3.0,0.0
1,2017-08-15 16:46:00,31011ade7c0a,12,3.0,0.0
2,2017-08-15 16:47:00,31011ade7c0a,24,3.0,0.0
3,2017-08-15 16:48:00,31011ade7c0a,36,3.0,0.0
4,2017-08-15 16:49:00,31011ade7c0a,48,3.0,0.0


## find the changes

In [224]:
# Find the occurrences of changes in the 'awake' column
change_indices = new[new['awake'].diff().ne(0)].index

# Print the indices where the 'awake' column value changes
print(change_indices)

Index([    0,   296,   836,  1653,  2276,  3251,  3802,  4612,  5186,  6116,
        6611,  7428,  8111,  8913,  9513, 10342, 10965, 11885, 12457, 13327,
       13845, 14658, 15341],
      dtype='int64')


In [225]:
new['awake_changes'] = new['awake'].diff().ne(0).astype(int)
new.head(3)

,timestamp,series_id,step,awake,wearable_on,awake_changes
0,2017-08-15 16:45:00,31011ade7c0a,0,3.0,0.0,1
1,2017-08-15 16:46:00,31011ade7c0a,12,3.0,0.0,0
2,2017-08-15 16:47:00,31011ade7c0a,24,3.0,0.0,0


Just print the changes once to see

In [226]:
def print_changes(df):
    for index, row in df.iterrows():
        if row['awake_changes'] == 1:
            print(f"Row {index}:")
            print(f"  - Value of 'awake' for current row: {row['awake']}")
            if index > 0:
                previous_row = df.loc[index - 1]
                print(f"  - Value of 'awake' for previous row: {previous_row['awake']}")
            else:
                print("  - No previous row")
            print("--------------------------")

# Call the function
print_changes(new)


Row 0:
  - Value of 'awake' for current row: 3.0
  - No previous row
--------------------------
Row 296:
  - Value of 'awake' for current row: 0.0
  - Value of 'awake' for previous row: 3.0
--------------------------
Row 836:
  - Value of 'awake' for current row: 1.0
  - Value of 'awake' for previous row: 0.0
--------------------------
Row 1653:
  - Value of 'awake' for current row: 0.0
  - Value of 'awake' for previous row: 1.0
--------------------------
Row 2276:
  - Value of 'awake' for current row: 1.0
  - Value of 'awake' for previous row: 0.0
--------------------------
Row 3251:
  - Value of 'awake' for current row: 0.0
  - Value of 'awake' for previous row: 1.0
--------------------------
Row 3802:
  - Value of 'awake' for current row: 1.0
  - Value of 'awake' for previous row: 0.0
--------------------------
Row 4612:
  - Value of 'awake' for current row: 0.0
  - Value of 'awake' for previous row: 1.0
--------------------------
Row 5186:
  - Value of 'awake' for current row: 1.0


## add onset and wakeup

From the information of change and awake/asleep/not worn, i define possible canditates for onset and wakeup

In [241]:
def add_onset_add_wakeup(df):
    # Initialize a new column 'wakeup' with zeros
    df['wakeup'] = 0
    df['onset'] = 0

    for index, row in df.iterrows():
        if row['awake_changes'] == 1:
            if index > 0:
                previous_row = df.loc[index - 1]
                # Check condition for setting 'onset' column
                if row['awake'] == 0.0 and previous_row['awake'] == 1.0:
                    df.at[index, 'onset'] = 1
                
                # Check condition for setting 'wakeup' column
                if row['awake'] == 1.0 and previous_row['awake'] == 0.0:
                    df.at[index, 'wakeup'] = 1

# Call the function
add_onset_add_wakeup(new)


In [242]:
new.head()

,timestamp,series_id,step,awake,wearable_on,awake_changes,wakeup,onset
0,2017-08-15 16:45:00,31011ade7c0a,0,3.0,0.0,1,0,0
1,2017-08-15 16:46:00,31011ade7c0a,12,3.0,0.0,0,0,0
2,2017-08-15 16:47:00,31011ade7c0a,24,3.0,0.0,0,0,0
3,2017-08-15 16:48:00,31011ade7c0a,36,3.0,0.0,0,0,0
4,2017-08-15 16:49:00,31011ade7c0a,48,3.0,0.0,0,0,0


In [244]:
new[new['onset'] == 1]

,timestamp,series_id,step,awake,wearable_on,awake_changes,wakeup,onset
1653,2017-08-16 20:18:00,31011ade7c0a,19836,0.0,1.0,1,0,1
3251,2017-08-17 22:56:00,31011ade7c0a,39012,0.0,1.0,1,0,1
4612,2017-08-18 21:37:00,31011ade7c0a,55344,0.0,1.0,1,0,1
6116,2017-08-19 22:41:00,31011ade7c0a,73392,0.0,1.0,1,0,1
7428,2017-08-20 20:33:00,31011ade7c0a,89136,0.0,1.0,1,0,1
8913,2017-08-21 21:18:00,31011ade7c0a,106956,0.0,1.0,1,0,1
10342,2017-08-22 21:07:00,31011ade7c0a,124104,0.0,1.0,1,0,1
11885,2017-08-23 22:50:00,31011ade7c0a,142620,0.0,1.0,1,0,1
13327,2017-08-24 22:52:00,31011ade7c0a,159924,0.0,1.0,1,0,1
14658,2017-08-25 21:03:00,31011ade7c0a,175896,0.0,1.0,1,0,1


## only keep sleeping windows longer than 30 min

## if the break between two sleeping window is smaller than 30 min count it as one

## only keep the longest sleeping window of one night

## add a curve around the window with percentage